In [1]:
import numpy as np
import time

In [2]:
#@np.vectorize
def deposit_particles(position, charge, mins, maxs, sizes):
    '''
    mins  = (xmin, ymin, zmin)
    maxs  = (xmax, ymax, zmax)
    sizes = (Nx, Ny, Nz)
    '''

    if ((mins.shape != maxs.shape) | (mins.shape != sizes.shape)):
        print('Dimension mismatch between mins, maxs, and sizes!!')
        return None
    
    if (np.any( position - mins < 0 ) | np.any( position - maxs >= 0 )):
        print(np.any( position - mins < 0 ) )
        print(np.any( position - maxs >= 0 ) )
        print('Particle position falls outside the boundary specified!')
        return None
    
    deltas = (maxs - mins) / (sizes - 1)
        #print('deltas: ',deltas)
    floors = np.floor( (position - mins)/deltas + 1 )  # index of the nearest "floor point"
    floors = floors.astype(int)
        #print('floors: ', floors)
    weights = ((mins - position) + floors*deltas) / deltas  # werights towards the floor point
        #print('weights: ', weights)
    
    dim = sizes.shape[0]
    if (dim > 3) :
        print('Dimension > 3 detected!!')
        return None

    elif (dim == 3):

        ip = floors[:,0]
        jp = floors[:,1]
        kp = floors[:,2]
        
        w1 = weights[:,0]
        w2 = weights[:,1]
        w3 = weights[:,2]        
 
        t1 = w1*w2*w3*charge
        t2 = w1*(1-w2)*w3*charge
        t3 = w1*(1-w2)*(1-w3)*charge
        t4 = w1*w2*(1-w3)*charge

        t5 = (1-w1)*w2*w3*charge
        t6 = (1-w1)*(1-w2)*w3*charge
        t7 = (1-w1)*(1-w2)*(1-w3)*charge
        t8 = (1-w1)*w2*(1-w3)*charge

        indexes = np.array([ip,jp,kp])
        contrib = np.array([t1,t2,t3,t4,t5,t6,t7,t8])
        
    elif (dim == 2):
        ip = floors[:,0]
        jp = floors[:,1]
        
        w1 = weights[:,0]
        w2 = weights[:,1]     
 
        t1 = w1*w2*charge
        t2 = w1*(1-w2)*charge
        t3 = (1-w1)*(1-w2)*charge
        t4 = (1-w1)*w2*charge

        indexes = np.array([ip,jp])
        contrib = np.array([t1,t2,t3,t4])
    
    elif (dim == 1):

        ip = floors[:,0]    
        w1 = weights[:,0]   
 
        t1 = w1*charge
        t2 = (1-w1)*charge

        indexes = np.array([ip])
        contrib = np.array([t1,t2])
        
    return indexes, contrib

In [3]:
np.random.seed(0)

In [4]:
charge = 25
mins = np.array([-0.2,-0.2,-0.2])  # Lower bounds of the grid
maxs = np.array([0.2,0.2,0.2])     # Upper bounds of the grid
sizes = np.array([500,500,500])

#beam1 = np.array([[1,1,1]])
#beam2 = np.array([[0.1,0.1,0.1]])
#beam3 = np.array([[1,1,1],[0.1,0.1,0.1],[2.3,2.5,2.9]])

beam4 = np.random.normal(0, 0.02, (1000000, 3))

Np = beam4.shape[0]

In [5]:
indexes, contrib = deposit_particles(beam4, charge, mins, maxs, sizes)

In [ ]:
tt1 = time.time();
# Method 1

charge_grid = np.zeros(sizes)
# Populate charge_grid
for n in range(Np):
    (ip, jp, kp) = indexes[:,n] # depositting index of the nth particle
    (t1, t2, t3, t4, t5, t6, t7, t8) = contrib[:,n] # contribtuion of the nth particle
    
    charge_grid[ip-1][jp-1][kp-1] = charge_grid[ip-1][jp-1][kp-1] + t1
    charge_grid[ip-1][jp][kp-1]   = charge_grid[ip-1][jp][kp-1]   + t2
    charge_grid[ip-1][jp][kp]     = charge_grid[ip-1][jp][kp]     + t3
    charge_grid[ip-1][jp-1][kp]   = charge_grid[ip-1][jp-1][kp]   + t4

    charge_grid[ip][jp-1][kp-1]   = charge_grid[ip][jp-1][kp-1] + t5
    charge_grid[ip][jp][kp-1]     = charge_grid[ip][jp][kp-1]   + t6
    charge_grid[ip][jp][kp]       = charge_grid[ip][jp][kp]     + t7
    charge_grid[ip][jp-1][kp]     = charge_grid[ip][jp-1][kp]   + t8

#charge_grid[50][56][50];

tt2 = time.time();

print('time:', tt2 - tt1)

In [ ]:
charge_grid[250][250][250]

In [ ]:
charge_grid[250:254,250:254,250:254]

In [11]:
y = np.random.random((500, 500, 500))
y.shape

(500, 500, 500)

In [12]:
# this saves memory
def hist_grid(sizes,indexes,contrib):
    
    xedges = np.arange(sizes[0]+1)
    yedges = np.arange(sizes[1]+1)
    zedges = np.arange(sizes[2]+1)

    h1 = np.sum( [ 
     np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges+1), weights=contrib[0])[0],
     np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges+1), weights=contrib[1])[0],
     np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges), weights=contrib[2])[0],
     np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges), weights=contrib[3])[0],

     np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges+1), weights=contrib[4])[0],
     np.histogramdd(indexes.T, bins = (xedges,yedges,zedges+1), weights=contrib[5])[0],
     np.histogramdd(indexes.T, bins = (xedges,yedges,zedges), weights=contrib[6])[0],
     np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges), weights=contrib[7])[0]])

    return h1


In [13]:
tt1 = time.time();
hist_grid(sizes,indexes,contrib)

tt2 = time.time();
print('time:', tt2 - tt1)

time: 21.767561674118042


In [6]:
tt1 = time.time();

xedges = np.arange(sizes[0]+1)
yedges = np.arange(sizes[1]+1)
zedges = np.arange(sizes[2]+1)

h1 = np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges+1), weights=contrib[0])[0]
h2 = np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges+1), weights=contrib[1])[0]
h3 = np.histogramdd(indexes.T, bins = (xedges+1,yedges,zedges), weights=contrib[2])[0]
h4 = np.histogramdd(indexes.T, bins = (xedges+1,yedges+1,zedges), weights=contrib[3])[0]

h5 = np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges+1), weights=contrib[4])[0]
h6 = np.histogramdd(indexes.T, bins = (xedges,yedges,zedges+1), weights=contrib[5])[0]
h7 = np.histogramdd(indexes.T, bins = (xedges,yedges,zedges), weights=contrib[6])[0]
h8 = np.histogramdd(indexes.T, bins = (xedges,yedges+1,zedges), weights=contrib[7])[0]

htot = h1+h2+h3+h4+h5+h6+h7+h8
htot[50][56][50];

tt2 = time.time();
print('time:', tt2 - tt1)

time: 11.502238750457764


In [ ]:
import concurrent.futures as cf

In [ ]:
# parallel?
with concurrent.futures.ProcessPoolExecutor(max_workers = 24) as executor:
    temp = executor.map(psi_s, zm2, xm2, beta_grid)
    result2 = np.array(list(temp))

In [ ]:
htot[250:254,250:254,250:254]

In [ ]:
htot[48:52,48:52,48:52]

In [ ]:
r = np.ones((4,3))
np.histogramdd(r, bins = (3,3,3), weights=(0.1,0.3,0.5,0.7))

In [ ]:
xedges = [0, 1,2,3,4,5]
yedges = [0, 1,2,3,4,5]
np.histogram2d([1,1,2,3,4,4],[1,2,2,3,4,4],bins = (xedges, yedges), weights=[1,2,3,4,5,6])

In [ ]:
np.histogram([1, 2, 1], bins=[0, 1, 2, 3])

In [ ]:
np.histogram([[1, 2, 1], [1, 0, 1]], bins=[0,1,2,3])

In [ ]:
np.histogram([1,2,3,4,5,6,7,7,7], bins=7)